<a href="https://colab.research.google.com/github/Larsdegroot/med_Unet/blob/lars_eval_fixing/run_experiment_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run UNet Training
This notebook is meant to be used in google colab to train a UNet model on the WMH dataset for MRI brain image segmentation.

## Set up environment

In [1]:
# don't restart session it's fine colab is a liar
!pip3 install torch torchvision torchaudio
!pip3 install "lightning[pytorch-extra]"
!pip3 install monai
!pip3 install SimpleITK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.4/624.4 kB 33.4 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 9.7 MB/s eta 0:00:00


In [6]:
#### To get a specific branch add: git clone -b <branch_name> <"url to github">
! git clone -b lars_eval_fixing "https://github.com/Larsdegroot/med_Unet"

fatal: destination path 'med_Unet' already exists and is not an empty directory.


In [29]:
# Get the latest version (usefull during development)
! git -C med_Unet/ pull origin lars_eval_fixing

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 458 bytes | 458.00 KiB/s, done.
From https://github.com/Larsdegroot/med_Unet
 * branch            lars_eval_fixing -> FETCH_HEAD
   af76829..3002c45  lars_eval_fixing -> origin/lars_eval_fixing
Updating af76829..3002c45
Fast-forward
 model.py | 32 ++++++++++++++++++--------------
 1 file changed, 18 insertions(+), 14 deletions(-)


In [4]:
import zipfile
from pathlib import Path

# Define the target directory
target_dir = Path("med_Unet/data")

# Create the directory if it doesn't exist
target_dir.mkdir(parents=True, exist_ok=True)

# Extract the ZIP files
for dataset in ["WMH", "BraTS"]:
  try:
    with zipfile.ZipFile(f"med_Unet/data/{dataset}.zip", 'r') as zip_ref:
        zip_ref.extractall(target_dir)
    print(f"Extracted the {dataset} dataset.")
  except FileNotFoundError: print(f"No zipfile found for the {dataset} dataset in {target_dir}")
  except zipfile.BadZipFile: print(f"Zipfile for {dataset} not complete, wait for it to upload")


Extracted the WMH dataset.
No zipfile found for the BraTS dataset in med_Unet/data


## Run training

In [30]:
!python3 med_Unet/main.py fit --config med_Unet/configs/default_config_2.yaml

2025-01-12 18:22:33.425978: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-12 18:22:33.446058: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-12 18:22:33.452668: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-12 18:22:33.467602: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-12 18:22:34.578106: W tensorflow/comp

## Run validation

In [20]:
!python3 med_Unet/main.py validate --config med_Unet/configs/default_config_2.yaml

2025-01-12 15:28:51.067140: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-12 15:28:51.102693: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-12 15:28:51.113290: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-12 15:28:51.136958: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-12 15:28:52.655157: W tensorflow/comp

## Run Testing

In [ ]:
#!python3 med_Unet/main.py test --config med_Unet/configs/testing_config